In [0]:
#Instalação das bibliotecas necessárias

from pyspark.sql import functions as F
from pyspark.sql.types import DateType, TimestampType

In [0]:
%sql
select * from tb_votacoes_bronze;

In [0]:

df_votacoes_bronze = spark.table("tb_votacoes_bronze") # Carrega a tabela bronze

# Transformações
df_votacoes_silver = (
    df_votacoes_bronze
    .withColumn("data", F.to_date("data", "yyyy-MM-dd"))
    .withColumn("dataHoraRegistro", F.to_timestamp("dataHoraRegistro"))
    .withColumn("descricao", F.initcap(F.trim("descricao")))
    .withColumn(
        "resultado_votacao",
        F.when(F.col("aprovacao") == 1, F.lit("Aprovada"))
         .when(F.col("aprovacao") == 0, F.lit("Rejeitada"))
         .otherwise(F.lit("Desconhecida")))
    .withColumn("siglaOrgao", F.upper(F.trim("siglaOrgao")))
    .withColumn("data_ingestao_silver", F.current_timestamp()) # adiciona coluna de controle da camada silver
)

# Salva a tabela em Delta 
df_votacoes_silver.write.format("delta").mode("overwrite").saveAsTable("tb_votacoes_silver")

print("Tabela 'tb_votacoes_silver' criada/atualizada com sucesso!")


In [0]:
%sql
select * from tb_votacoes_silver;